In [10]:
# https://juliadatacubes.github.io/YAXArrays.jl/dev/examples/generated/UserGuide/examples_from_esdl_study_4/
using Pkg

# for operating data cubes
using Zarr, YAXArrays, NetCDF
using DimensionalData
using EarthDataLab

# for data analysis
using WeightedOnlineStats
using Statistics, Dates, SkipNan
using DataFrames, Downloads

Base.names(ds::Dataset) = string.(collect(keys(ds.cubes)))
Base.getindex(ds::Dataset, i) = ds[names(ds)[i]]
chunksize(cube) = Cubes.cubechunks(cube)

chunksize (generic function with 1 method)

# 1. 读取数据

In [12]:
# import Pkg; Pkg.add("SkipNan")

In [13]:
@time CUBE = esdc(res="tiny")

 24.805153 seconds (34.08 k allocations: 3.251 MiB)


144×72×989×42 YAXArray{Union{Missing, Float32},4} with dimensions: 
  Dim{:lon} Sampled{Float64} -178.75:2.5:178.75 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} -88.75:2.5:88.75 ForwardOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[1979-01-09T00:00:00, …, 2021-12-27T00:00:00] ForwardOrdered Irregular Points,
  Dim{:Variable} Categorical{String} String[sensible_heat, potential_evaporation, …, net_ecosystem_exchange, snow_sublimation] Unordered
units: mm d^-1
Total size: 1.6 GB


In [26]:
vars = ["gross_primary_productivity", "surface_moisture"]
# col = CUBE[vars]
@time cube = CUBE[variable = At(vars), time = 2003:2012]
# gpp = cube_handle[variable = At("gross_primary_productivity"), time = 2003:2012]
# moisture = cube_handle[variable = At("surface_moisture"), time = 2003:2012]

  0.000376 seconds (286 allocations: 15.508 KiB)


144×72×230×2 YAXArray{Union{Missing, Float32},4} with dimensions: 
  Dim{:lon} Sampled{Float64} -178.75:2.5:178.75 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} -88.75:2.5:88.75 ForwardOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2003-01-09T00:00:00, …, 2012-12-26T00:00:00] ForwardOrdered Irregular Points,
  Dim{:Variable} Categorical{String} String["gross_primary_productivity", "surface_moisture"] Unordered
units: mm d^-1
Total size: 18.19 MB


In [27]:
f = "datacube-02"
@time savecube(cube, f, driver=:zarr) # YAXArray

Progress: 100%|█████████████████████████████████████████| Time: 0:00:23
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05


 49.729773 seconds (34.22 M allocations: 2.172 GiB, 9.34% gc time, 47.10% compilation time: <1% of which was recompilation)


144×72×230×2 YAXArray{Union{Missing, Float32},4} with dimensions: 
  Dim{:lon} Sampled{Float64} -178.75:2.5:178.75 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} -88.75:2.5:88.75 ForwardOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2003-01-09T00:00:00, …, 2012-12-26T00:00:00] ForwardOrdered Irregular Points,
  Dim{:Variable} Categorical{String} String["gross_primary_productivity", "surface_moisture"] ForwardOrdered
units: mm d^-1
Total size: 18.19 MB


# 2. 数据分析

In [31]:
ds = YAXArrays.open_dataset(f)

moisture = ds[1]
gpp = ds[2]

144×72×230 YAXArray{Union{Missing, Float32},3} with dimensions: 
  Dim{:lon} Sampled{Float64} -178.75:2.5:178.75 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} -88.75:2.5:88.75 ForwardOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2003-01-09T00:00:00, …, 2012-12-26T00:00:00] ForwardOrdered Irregular Points
name: gross_primary_productivity
units: mm d^-1
Total size: 9.1 MB


In [32]:
@time regions = Cube("https://s3.bgc-jena.mpg.de:9000/misc/IPCCregions_2d5.zarr")

  1.680891 seconds (2.76 k allocations: 299.484 KiB)


144×72 YAXArray{Union{Missing, Float32},2} with dimensions: 
  Dim{:lon} Sampled{Float64} -178.75:2.5:178.75 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} -88.75:2.5:88.75 ForwardOrdered Regular Points
units: ID
name: IPCCRegions
Total size: 40.5 KB


In [33]:
t = CubeTable(gpp=gpp, moisture=moisture, region=regions)

Datacube iterator with 1 subtables with fields: (:gpp, :moisture, :region, :lon, :lat, :Ti)

In [38]:
t[1]

Table chunk with schema:
Tables.Schema:
 :gpp       Union{Missing, Float32}
 :moisture  Union{Missing, Float32}
 :region    Union{Missing, Float32}
 :lon       Float64
 :lat       Float64
 :Ti        DateTime

In [39]:
using DataFrames, Base.Iterators
@time df = DataFrame(t[1])
# summary(df)
# first(df,5)

  0.606909 seconds (1.54 k allocations: 162.845 MiB, 6.67% gc time)


Row,gpp,moisture,region,lon,lat,Ti
,Float32?,Float32?,Float32?,Float64,Float64,DateTime
1,missing,0.0885,28.0,-178.75,-88.75,2003-01-09T00:00:00
2,missing,0.0885,28.0,-176.25,-88.75,2003-01-09T00:00:00
3,missing,0.0885,28.0,-173.75,-88.75,2003-01-09T00:00:00
4,missing,0.0885,28.0,-171.25,-88.75,2003-01-09T00:00:00
5,missing,0.0885,28.0,-168.75,-88.75,2003-01-09T00:00:00
6,missing,0.0885,28.0,-166.25,-88.75,2003-01-09T00:00:00
7,missing,0.0885,28.0,-163.75,-88.75,2003-01-09T00:00:00
8,missing,0.0885,28.0,-161.25,-88.75,2003-01-09T00:00:00
9,missing,0.0885,28.0,-158.75,-88.75,2003-01-09T00:00:00


In [40]:
# reigions' table with names, labels (abbreviations), ID, and coordinates
regions_df = ["region" "label" "id" "lon" "lat";
"Alaska/N.W. Canada" "ALA" 1 -136.5 60.3;
"Amazon" "AMZ" 2 -75.04506 -3.772654;
"Central America/Mexico" "CAM" 3 -96.24961 5.633803;
"small islands regions Caribbean" "CAR*" 4 -73.4605 16.31368;
"Central Asia" "CAS" 5 55 40
"Central Europe" "CEU" 6 5.74266 43.588774;
"Canada/Greenland/Iceland" "CGI" 7 -63.5 50.5;
"Central North America" "CNA" 8 -99 39.3
"East Africa" "EAF" 9 30.5 0;
"East Asia" "EAS" 10 105.5 35;
"East North America" "ENA" 11 -72.5 31;
"South Europe/Mediterranean" "MED" 12 10 33;
"North Asia" "NAS" 13 90 50;
"North Australia" "NAU" 14 122.5 -20;
"North-East Brazil" "NEB" 15 -56 -14;
"North Europe" "NEU" 16 12.27682 54.462817;
"Southern Africa" "SAF" 17 21 -23.2;
"Sahara" "SAH" 18 5 20;
"South Asia" "SAS" 19 70.58108 17.905405;
"South Australia/New Zealand" "SAU" 20 145 -40;
"Southeast Asia" "SEA" 21 115 5;
"Southeastern South America" "SSA" 22 -60.4941 -40.763987;
"Tibetan Plateau" "TIB" 23 77.5 30;
"West Africa" "WAF" 24 2.5 1.8;
"West Asia" "WAS" 25 37 30;
"West North America" "WNA" 26 -131.5 44.3;
"West Coast South America" "WSA" 27 -75.88257 -30.783;
"Antarctica" "ANT*" 28 0 -70;
"Arctic" "ARC*" 29 -10 66.75;
"Pacific Islands region[2]" "NTP*" 30 182.5 15;
"Southern Topical Pacific" "STP*" 31 192.5 -15;
"Pacific Islands region[3]" "ETP*" 32 182.5 0;
"West Indian Ocean" "WIO*" 33 63.5 -10]

# subset solely the region labels for further use
labels_ipcc = regions_df[2:end,2]


33-element Vector{Any}:
 "ALA"
 "AMZ"
 "CAM"
 "CAR*"
 "CAS"
 "CEU"
 "CGI"
 "CNA"
 "EAF"
 "EAS"
 "ENA"
 "MED"
 "NAS"
 ⋮
 "SSA"
 "TIB"
 "WAF"
 "WAS"
 "WNA"
 "WSA"
 "ANT*"
 "ARC*"
 "NTP*"
 "STP*"
 "ETP*"
 "WIO*"

In [ ]:
function weightedhistograms_by_region(df, labels)
    hists = [WeightedHist((0.0:0.1:12,0:0.01:1)) for _ in eachindex(labels_ipcc)]
    for row in Tables.rows(df)
        # if all data are there
        if !any(ismissing, (row.gpp, row.moisture, row.region))
            # we select the appropriate histogram according to the region the data point belongs to
            h = hists[Int(row.region)]
            # and we fit the two data points to the histogram, weight by cos of lat
            fit!(h, (row.gpp, row.moisture), cosd(row.lat))
        end
    end

    # we create the axes for the new output data cube
    midpointsgpp   = 0.05:0.1:11.95
    midpointsmoist = 0.005:0.01:0.995
    newaxes = (
        Dim{:SREX}([labels[i] for i in 1:33]),
        Dim{:GPP}(midpointsgpp),
        Dim{:Moisture}(midpointsmoist)
        )
    # and create the new cube object
    data = [WeightedOnlineStats.pdf(hists[reg],(g,m)) for reg in 1:33, g in midpointsgpp, m in midpointsmoist]
    data = replace(data,NaN => missing, 0 => missing)
    YAXArray(newaxes, data)
end

r = weightedhistograms_by_region(df, labels_ipcc);

rdata = mapslices(r,dims=("GPP","Moisture")) do xin
    xin ./ maximum(skipmissing(xin))
end
